In [1]:
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats

In [2]:
START = 2002
END = 2022

In [3]:
if os.path.exists("batting.csv"):
    batting = pd.read_csv("batting.csv", index_col=0)
else:
    batting = batting_stats(START, END, qual=200)
    batting.to_csv("batting.csv")

In [4]:
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)

In [5]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
8,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
6535,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
6673,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
6988,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


In [6]:
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

C:\Users\jaymi\AppData\Local\Temp\ipykernel_20072\1658847979.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


In [7]:
batting[["Name","Season","WAR","Next_WAR"]]

,Name,Season,WAR,Next_WAR
5562,Alfredo Amezaga,2006,1.1,2.0
5006,Alfredo Amezaga,2007,2.0,1.2
5252,Alfredo Amezaga,2008,1.2,NaN
1169,Garret Anderson,2002,3.7,5.1
864,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6002,Owen Miller,2022,0.7,NaN
4881,Andrew Vaughn,2021,-0.2,-0.5
3377,Andrew Vaughn,2022,-0.5,NaN
6620,Ha-seong Kim,2021,0.4,3.6


In [8]:
null_count = batting.isnull().sum()


In [9]:
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
xBA         6754
xSLG        6754
xwOBA       6754
L-WAR          0
Next_WAR    1179
Length: 321, dtype: int64

In [10]:
complete_cols = list(batting.columns[null_count == 0])


In [11]:
batting = batting[complete_cols + ["Next_WAR"]].copy()


In [12]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR
5562,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
5006,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
5252,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
1169,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,3.7,5.1
864,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,5.1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,24655,2022,Owen Miller,CLE,25,130,424,472,103,70,...,111,97,131,100,83,340,0.188,0.266,0.7,NaN
4881,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,104,116,84,99,110,321,0.185,0.285,-0.4,-0.5
3377,26197,2022,Andrew Vaughn,CHW,24,134,510,555,138,92,...,106,111,94,100,104,419,0.201,0.291,-0.5,NaN
6620,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,99,59,137,96,88,201,0.216,0.303,0.5,3.6


In [13]:
batting.dtypes

IDfg          int64
Season        int64
Name         object
Team         object
Age           int64
             ...   
Events        int64
CStr%       float64
CSW%        float64
L-WAR       float64
Next_WAR    float64
Length: 133, dtype: object

In [14]:
batting.dtypes[batting.dtypes == "object"]


Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [15]:
del batting["Age Rng"]
del batting["Dol"]

In [16]:
batting["team_code"] = batting["Team"].astype("category").cat.codes


In [17]:
batting_full = batting.copy()
batting = batting.dropna().copy()

In [18]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=4
                               )

In [19]:
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

In [20]:
from sklearn.preprocessing import MinMaxScaler

# Ensure selected columns are of float type
batting[selected_columns] = batting[selected_columns].astype(float)

# Apply MinMaxScaler
scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

In [21]:
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code
count,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,...,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000
mean,5366.785830,2011.163229,0.360600,0.652755,0.478666,0.480943,0.365973,0.290481,0.399279,0.103459,...,0.403164,0.410923,0.511026,0.478646,0.172991,0.498932,0.545898,0.322040,1.793130,0.474128
std,5133.255295,5.612014,0.147476,0.255929,0.242481,0.262290,0.182585,0.138786,0.171732,0.105891,...,0.131213,0.121082,0.130359,0.133992,0.273858,0.137180,0.120701,0.122153,1.981035,0.305105
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000
25%,1131.500000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,0.043478,...,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.234177,0.350000,0.205882
50%,3531.000000,2011.000000,0.346154,0.709402,0.505396,0.508651,0.370690,0.283019,0.379310,0.086957,...,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,0.303797,1.500000,0.470588
75%,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.710208,0.508621,0.391509,0.517241,0.130435,...,0.488722,0.483146,0.594203,0.564626,0.346411,0.591489,0.625551,0.392405,2.900000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [22]:
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [23]:
predictors = list(selected_columns[sfs.get_support()])


In [24]:
predictors

['Age',
 'IBB',
 'SO',
 'SB',
 'IFH',
 'BU',
 'BABIP',
 'WAR',
 'Spd',
 'WPA',
 'PH',
 'CB%',
 'CH%',
 'wCH',
 'Oppo%',
 'OBP+',
 'SLG+',
 'Pull%+',
 'Soft%+',
 'Hard%+']

In [25]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []
    
    years = sorted(data["Season"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]
        
        model.fit(train[predictors], train["Next_WAR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Next_WAR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [26]:
predictions = backtest(batting, rr, predictors)


In [27]:
predictions

,actual,prediction
5006,1.2,1.405835
1925,1.4,0.716105
3102,-0.1,0.457908
5797,0.6,0.979155
1109,4.8,2.214873
...,...,...
1914,2.2,2.752087
5875,0.8,2.084895
7032,0.7,1.583738
4881,-0.5,1.819861


In [28]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

2.7363675228708013

In [29]:
batting["Next_WAR"].describe()

count    5575.000000
mean        1.793130
std         1.981035
min        -3.100000
25%         0.350000
50%         1.500000
75%         2.900000
max        11.900000
Name: Next_WAR, dtype: float64

In [30]:
def player_history(df):
    df = df.sort_values("Season").copy()  # Create a copy to avoid warnings
    
    # Add 'player_season' column
    df["player_season"] = range(0, df.shape[0])
    
    # Calculate correlation and fill NaN values
    expanding_corr = df[["player_season", "WAR"]].expanding().corr()
    df["war_corr"] = expanding_corr.loc[(slice(None), "player_season"), "WAR"].values
    df["war_corr"] = df["war_corr"].fillna(0)
    
    # Calculate WAR difference and handle NaN and inf
    df["war_diff"] = df["WAR"] / df["WAR"].shift(1)
    df["war_diff"] = df["war_diff"].fillna(1).replace(np.inf, 1)
    # df["war_diff"].fillna(1, inplace=True)
    # df["war_diff"][df["war_diff"] == np.inf] = 1
    
    return df

batting = batting.groupby("IDfg", group_keys=False).apply(player_history)

C:\Users\jaymi\AppData\Local\Temp\ipykernel_20072\3490469165.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(player_history)


In [31]:
def group_averages(df):
    return df["WAR"] / df["WAR"].mean()

In [32]:
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)


C:\Users\jaymi\AppData\Local\Temp\ipykernel_20072\1513893542.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)


In [33]:
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]


In [34]:
predictions = backtest(batting, rr, new_predictors)


In [35]:
predictions

,actual,prediction
5006,1.2,1.328440
1925,1.4,0.632482
3102,-0.1,0.368192
5797,0.6,1.069567
1109,4.8,2.055862
...,...,...
1914,2.2,2.696359
5875,0.8,2.049233
7032,0.7,1.441442
4881,-0.5,1.644603


In [36]:
mean_squared_error(predictions["actual"], predictions["prediction"]) 


2.6815953722706385

In [37]:
pd.Series(rr.coef_, index=new_predictors).sort_values()


Age             -2.687734
BABIP           -1.971640
WAR             -1.961713
SLG+            -1.404307
Soft%+          -1.322527
BU              -1.146788
SO              -0.859099
PH              -0.738177
WPA             -0.578544
CB%             -0.324474
wCH             -0.287639
war_diff        -0.281043
CH%             -0.253139
Pull%+          -0.228693
war_corr        -0.128594
player_season    0.009565
IFH              0.661687
Oppo%            0.686142
Spd              0.756137
OBP+             0.824955
SB               0.980457
IBB              2.084161
Hard%+           2.433063
war_season       3.242329
dtype: float64

In [38]:
diff = predictions["actual"] - predictions["prediction"]


In [39]:
merged = predictions.merge(batting, left_index=True, right_index=True)


In [40]:
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()


In [41]:
merged[["IDfg", "Season", "Name", "WAR", "Next_WAR", "diff"]].sort_values(["diff"])


,IDfg,Season,Name,WAR,Next_WAR,diff
6738,5343,2011,Brandon Crawford,0.221519,1.9,0.000004
926,1679,2010,Chase Utley,0.531646,4.0,0.000250
3879,1591,2008,Jamey Carroll,0.310127,1.3,0.000341
6223,6104,2015,Aaron Hill,0.189873,0.8,0.001793
3400,3882,2007,Chris Young,0.246835,2.3,0.001951
...,...,...,...,...,...,...
871,9166,2010,Buster Posey,0.443038,9.8,6.466616
3823,1875,2009,Josh Hamilton,0.278481,8.4,6.497212
3245,5631,2010,Matt Kemp,0.196203,8.3,6.552455
451,15640,2021,Aaron Judge,0.544304,11.1,7.329714
